In [19]:
import yfinance as yf
import pandas as pd
apple = yf.Ticker("MMM")
apple = apple.history(period="max")
del apple["Dividends"]
del apple["Stock Splits"]
apple["Tomorrow"] = apple["Close"].shift(-1)
apple["Target"] = (apple["Tomorrow"] > apple["Close"]).astype(int)
apple = apple.loc["1990-01-01":].copy()

file_path = '~/Documents/CS 5100/Stock-Trends/data/news/sentiments/general_scores.csv'
df = pd.read_csv(file_path)
del df['Avg_Polarity']
del df['Avg_Subjectivity']

def predict(train, test, predictors, model):
    model.fit(train[predictors], train["Target"])
    preds = model.predict(test[predictors])
    preds = pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test["Target"], preds], axis=1)
    return combined

apple_copy = apple.copy()
apple_copy.reset_index(inplace=True)
apple_copy['Date'] = apple_copy['Date'].apply(lambda x: x.strftime('%Y-%m-%d'))
merged_data = pd.merge(apple_copy, df, on='Date', how='outer')
merged_data.set_index('Date', inplace=True)
merged_data = merged_data.dropna()

def backtest(data, model, predictors, start=300, step=50):
    all_predictions = []

    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+step)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)
    
    return pd.concat(all_predictions)

In [20]:
df

,Date,Avg_Pos_Score,Avg_Neu_Score,Avg_Neg_Score
0,2019-01-01,0.038333,0.894889,0.066778
1,2019-01-02,0.073750,0.848000,0.078250
2,2019-01-03,0.057667,0.906111,0.036222
3,2019-01-04,0.069222,0.833000,0.097889
4,2019-01-07,0.056111,0.815444,0.128556
...,...,...,...,...
779,2021-12-27,0.056300,0.831000,0.112700
780,2021-12-28,0.114800,0.786600,0.098700
781,2021-12-29,0.040700,0.820000,0.139400
782,2021-12-30,0.043600,0.876700,0.079700


In [21]:
merged_data

,Open,High,Low,Close,Volume,Tomorrow,Target,Avg_Pos_Score,Avg_Neu_Score,Avg_Neg_Score
Date,,,,,,,,,,
2019-01-02,121.014834,123.057303,120.293210,123.031540,2960339.0,118.398972,0.0,0.073750,0.848000,0.078250
2019-01-03,121.311256,121.311256,117.838413,118.398972,4016407.0,123.269951,1.0,0.057667,0.906111,0.036222
2019-01-04,120.325448,123.695205,119.861533,123.269951,3582140.0,122.986458,0.0,0.069222,0.833000,0.097889
2019-01-07,123.295727,123.901379,121.556084,122.986458,2585991.0,123.501900,1.0,0.056111,0.815444,0.128556
2019-01-08,124.352384,125.067577,122.148838,123.501900,2965841.0,123.901375,1.0,0.040000,0.874100,0.085900
...,...,...,...,...,...,...,...,...,...,...
2021-12-27,127.345534,128.494302,127.171039,128.472488,1414868.0,129.155945,1.0,0.056300,0.831000,0.112700
2021-12-28,128.574289,129.781227,128.508862,129.155945,1936204.0,129.715775,1.0,0.114800,0.786600,0.098700
2021-12-29,129.039607,130.064765,128.857841,129.715775,1461392.0,129.155945,0.0,0.040700,0.820000,0.139400


In [22]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=200, min_samples_split=50, random_state=1)
predictors = ["Close", "Volume", "Open", "High", "Low"]
new_predictors = ["Close", "Volume", "Open", "High", "Low", "Avg_Pos_Score", "Avg_Neu_Score", "Avg_Neg_Score"]

predictions = backtest(merged_data, model, predictors)
new_predictions = backtest(merged_data, model, new_predictors)

precision_score(predictions["Target"], predictions["Predictions"])

0.5523465703971119

In [23]:
precision_score(new_predictions["Target"], new_predictions["Predictions"])

0.5636363636363636